# 3-Model Ensemble: MLP[32,16] + LightGBM + MLP[64,32]

**Rationale**: exp_012 (2-model ensemble) achieved LB 0.0913, beating [32,16] alone (0.0932) by 2.04%.
Adding a third diverse model might improve further.

**Ensemble composition**:
- MLP[32,16]: weight 0.5 (best LB model)
- LightGBM: weight 0.3 (different model family)
- MLP[64,32]: weight 0.2 (different architecture, different error patterns)

**Baseline to beat**: exp_012 CV 0.009004, LB 0.0913

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import lightgbm as lgb
import tqdm
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.set_default_dtype(torch.double)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cpu


In [2]:
# Data loading
DATA_PATH = '/home/data'
INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[["Residence Time", "Temperature", "SOLVENT NAME"]]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

# Load features
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
DRFP_DF = pd.read_csv(f'{DATA_PATH}/drfps_catechol_lookup.csv', index_col=0)
drfp_variance = DRFP_DF.var()
nonzero_variance_cols = drfp_variance[drfp_variance > 0].index.tolist()
DRFP_FILTERED = DRFP_DF[nonzero_variance_cols]

print(f'Spange: {SPANGE_DF.shape}, DRFP filtered: {DRFP_FILTERED.shape}')

Spange: (26, 13), DRFP filtered: (24, 122)


In [3]:
# Featurizer with Arrhenius kinetics
class CombinedFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.drfp_df = DRFP_FILTERED

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            A_drfp = self.drfp_df.loc[X["SOLVENT A NAME"]].values
            B_drfp = self.drfp_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
                X_drfp = B_drfp * (1 - (1-pct)) + A_drfp * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
                X_drfp = A_drfp * (1 - pct) + B_drfp * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
            X_drfp = self.drfp_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange, X_drfp])
    
    def featurize_torch(self, X, flip=False):
        return torch.tensor(self.featurize(X, flip))

print('Featurizer defined')

Featurizer defined


In [4]:
# MLP Model with configurable architecture
class MLPModelInternal(nn.Module):
    def __init__(self, input_dim, hidden_dims=[32, 16], output_dim=3, dropout=0.05):
        super(MLPModelInternal, self).__init__()
        layers = [nn.BatchNorm1d(input_dim)]
        prev_dim = input_dim
        for h_dim in hidden_dims:
            layers.extend([nn.Linear(prev_dim, h_dim), nn.BatchNorm1d(h_dim), nn.ReLU(), nn.Dropout(dropout)])
            prev_dim = h_dim
        layers.extend([nn.Linear(prev_dim, output_dim), nn.Sigmoid()])
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

class MLPEnsemble:
    def __init__(self, hidden_dims=[32, 16], n_models=5, data='single'):
        self.hidden_dims = hidden_dims
        self.n_models = n_models
        self.data_type = data
        self.featurizer = CombinedFeaturizer(mixed=(data=='full'))
        self.models = []

    def train_model(self, X_train, y_train, epochs=200, batch_size=32, lr=5e-4):
        X_std = self.featurizer.featurize_torch(X_train, flip=False)
        y_vals = torch.tensor(y_train.values)
        
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize_torch(X_train, flip=True)
            X_all = torch.cat([X_std, X_flip], dim=0)
            y_all = torch.cat([y_vals, y_vals], dim=0)
        else:
            X_all, y_all = X_std, y_vals
            
        input_dim = X_all.shape[1]
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        for i in range(self.n_models):
            torch.manual_seed(42 + i * 13)
            np.random.seed(42 + i * 13)
            model = MLPModelInternal(input_dim, hidden_dims=self.hidden_dims, dropout=0.05).to(device)
            model.train()
            self.models.append(model)
            
            dataset = TensorDataset(X_all, y_all)
            loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
            optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
            criterion = nn.HuberLoss()
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20)
            
            for epoch in range(epochs):
                epoch_loss = 0.0
                for inputs, targets in loader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs, targets)
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                    optimizer.step()
                    epoch_loss += loss.item() * inputs.size(0)
                scheduler.step(epoch_loss / len(dataset))

    def predict(self, X):
        device = next(self.models[0].parameters()).device
        if self.data_type == 'full':
            X_std = self.featurizer.featurize_torch(X, flip=False).to(device)
            X_flip = self.featurizer.featurize_torch(X, flip=True).to(device)
            pred_sum = torch.zeros((len(X), 3)).to(device)
            with torch.no_grad():
                for model in self.models:
                    model.eval()
                    pred_sum += (model(X_std) + model(X_flip)) * 0.5
            return (pred_sum / self.n_models).cpu().numpy()
        else:
            X_std = self.featurizer.featurize_torch(X).to(device)
            pred_sum = torch.zeros((len(X), 3)).to(device)
            with torch.no_grad():
                for model in self.models:
                    model.eval()
                    pred_sum += model(X_std)
            return (pred_sum / self.n_models).cpu().numpy()

print('MLPEnsemble defined')

MLPEnsemble defined


In [5]:
# LightGBM Model
class LGBMModel:
    def __init__(self, data='single'):
        self.data_type = data
        self.featurizer = CombinedFeaturizer(mixed=(data=='full'))
        self.models = []
        self.params = {'objective': 'regression', 'metric': 'mse', 'boosting_type': 'gbdt',
                       'num_leaves': 31, 'learning_rate': 0.05, 'feature_fraction': 0.8,
                       'bagging_fraction': 0.8, 'bagging_freq': 5, 'verbose': -1, 'seed': 42}

    def train_model(self, X_train, y_train, num_boost_round=200):
        X_feat = self.featurizer.featurize(X_train)
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize(X_train, flip=True)
            X_all = np.vstack([X_feat, X_flip])
            y_all = np.vstack([y_train.values, y_train.values])
        else:
            X_all, y_all = X_feat, y_train.values
        
        self.models = []
        for i in range(3):
            train_data = lgb.Dataset(X_all, label=y_all[:, i])
            model = lgb.train(self.params, train_data, num_boost_round=num_boost_round)
            self.models.append(model)

    def predict(self, X):
        if self.data_type == 'full':
            X_std = self.featurizer.featurize(X, flip=False)
            X_flip = self.featurizer.featurize(X, flip=True)
            preds = np.zeros((len(X), 3))
            for i, model in enumerate(self.models):
                preds[:, i] = (model.predict(X_std) + model.predict(X_flip)) / 2
        else:
            X_feat = self.featurizer.featurize(X)
            preds = np.zeros((len(X), 3))
            for i, model in enumerate(self.models):
                preds[:, i] = model.predict(X_feat)
        return np.clip(preds, 0, 1)

print('LGBMModel defined')

LGBMModel defined


In [6]:
# 3-Model Ensemble: MLP[32,16] + LightGBM + MLP[64,32]
class ThreeModelEnsemble:
    """3-model ensemble for maximum diversity.
    
    Weights: MLP[32,16] 0.5, LightGBM 0.3, MLP[64,32] 0.2
    """
    def __init__(self, data='single'):
        self.data_type = data
        self.mlp_32_16 = None
        self.lgbm = None
        self.mlp_64_32 = None
        # Weights optimized for diversity
        self.w1 = 0.5  # MLP[32,16] - best LB model
        self.w2 = 0.3  # LightGBM - different model family
        self.w3 = 0.2  # MLP[64,32] - different architecture

    def train_model(self, X_train, y_train):
        # Train MLP[32,16] - best LB model
        self.mlp_32_16 = MLPEnsemble(hidden_dims=[32, 16], n_models=5, data=self.data_type)
        self.mlp_32_16.train_model(X_train, y_train, epochs=200)
        
        # Train LightGBM - different model family
        self.lgbm = LGBMModel(data=self.data_type)
        self.lgbm.train_model(X_train, y_train, num_boost_round=200)
        
        # Train MLP[64,32] - different architecture
        self.mlp_64_32 = MLPEnsemble(hidden_dims=[64, 32], n_models=5, data=self.data_type)
        self.mlp_64_32.train_model(X_train, y_train, epochs=200)

    def predict(self, X):
        pred1 = self.mlp_32_16.predict(X)
        pred2 = self.lgbm.predict(X)
        pred3 = self.mlp_64_32.predict(X)
        
        # Weighted average
        ensemble_pred = self.w1 * pred1 + self.w2 * pred2 + self.w3 * pred3
        return torch.tensor(np.clip(ensemble_pred, 0, 1))

print('ThreeModelEnsemble defined: MLP[32,16] (0.5) + LightGBM (0.3) + MLP[64,32] (0.2)')

ThreeModelEnsemble defined: MLP[32,16] (0.5) + LightGBM (0.3) + MLP[64,32] (0.2)


In [7]:
# Run CV for single solvent
print("\n--- TASK 0: Single Solvent (24 folds) ---")
X, Y = load_data("single_solvent")
all_preds_single = []
all_actuals_single = []

for fold_idx, split in tqdm.tqdm(enumerate(generate_leave_one_out_splits(X, Y)), total=24):
    (train_X, train_Y), (test_X, test_Y) = split
    model = ThreeModelEnsemble(data='single')
    model.train_model(train_X, train_Y)
    predictions = model.predict(test_X).numpy()
    all_preds_single.append(predictions)
    all_actuals_single.append(test_Y.values)

preds_single = np.vstack(all_preds_single)
actuals_single = np.vstack(all_actuals_single)
mse_single = np.mean((actuals_single - preds_single) ** 2)
print(f'\nSingle Solvent MSE: {mse_single:.6f}')


--- TASK 0: Single Solvent (24 folds) ---


  0%|          | 0/24 [00:00<?, ?it/s]

  4%|▍         | 1/24 [02:50<1:05:30, 170.89s/it]

  8%|▊         | 2/24 [05:28<59:50, 163.22s/it]  

 12%|█▎        | 3/24 [08:13<57:19, 163.76s/it]

...

 92%|█████████▏| 22/24 [53:02<03:58, 119.22s/it]

 96%|█████████▌| 23/24 [54:55<01:57, 117.54s/it]

100%|██████████| 24/24 [56:50<00:00, 116.58s/it]

100%|██████████| 24/24 [56:50<00:00, 142.09s/it]


Single Solvent MSE: 0.009436


In [8]:
# Run CV for full data
print("\n--- TASK 1: Full Data (13 folds) ---")
X, Y = load_data("full")
all_preds_full = []
all_actuals_full = []

for fold_idx, split in tqdm.tqdm(enumerate(generate_leave_one_ramp_out_splits(X, Y)), total=13):
    (train_X, train_Y), (test_X, test_Y) = split
    model = ThreeModelEnsemble(data='full')
    model.train_model(train_X, train_Y)
    predictions = model.predict(test_X).numpy()
    all_preds_full.append(predictions)
    all_actuals_full.append(test_Y.values)

preds_full = np.vstack(all_preds_full)
actuals_full = np.vstack(all_actuals_full)
mse_full = np.mean((actuals_full - preds_full) ** 2)
print(f'\nFull Data MSE: {mse_full:.6f}')


--- TASK 1: Full Data (13 folds) ---


  0%|          | 0/13 [00:00<?, ?it/s]

  8%|▊         | 1/13 [06:34<1:18:52, 394.38s/it]

 15%|█▌        | 2/13 [13:42<1:15:56, 414.24s/it]

 23%|██▎       | 3/13 [21:02<1:10:59, 425.92s/it]

...

 85%|████████▍ | 11/13 [1:30:38<18:23, 551.71s/it]

 92%|█████████▏| 12/13 [1:39:45<09:10, 550.51s/it]

100%|██████████| 13/13 [1:48:18<00:00, 539.08s/it]

100%|██████████| 13/13 [1:48:18<00:00, 499.90s/it]


Full Data MSE: 0.008783


In [9]:
# Calculate overall MSE
n_single = len(actuals_single)
n_full = len(actuals_full)
overall_mse = (mse_single * n_single + mse_full * n_full) / (n_single + n_full)

print(f'\n=== FINAL RESULTS ===')
print(f'Single Solvent MSE: {mse_single:.6f} (n={n_single})')
print(f'Full Data MSE: {mse_full:.6f} (n={n_full})')
print(f'Overall MSE: {overall_mse:.6f}')

print(f'\n=== COMPARISON ===')
print(f'exp_012 (2-model ensemble): CV 0.009004, LB 0.0913 (BEST LB)')
print(f'This 3-model ensemble: CV {overall_mse:.6f}')

if overall_mse < 0.009004:
    improvement = (0.009004 - overall_mse) / 0.009004 * 100
    print(f'\n✓ IMPROVEMENT: {improvement:.2f}% better than exp_012!')
    print(f'Predicted LB (using 10.14x ratio): {overall_mse * 10.14:.4f}')
else:
    degradation = (overall_mse - 0.009004) / 0.009004 * 100
    print(f'\n✗ WORSE: {degradation:.2f}% worse than exp_012')
    print(f'Keep exp_012 as best submission.')


=== FINAL RESULTS ===
Single Solvent MSE: 0.009436 (n=656)
Full Data MSE: 0.008783 (n=1227)
Overall MSE: 0.009011

=== COMPARISON ===
exp_012 (2-model ensemble): CV 0.009004, LB 0.0913 (BEST LB)
This 3-model ensemble: CV 0.009011

✗ WORSE: 0.07% worse than exp_012
Keep exp_012 as best submission.
